In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget
import numpy as np
import pandas as pd
import torch
import torchvision
from torch import optim
import matplotlib.pyplot as plt
import torch.nn.functional as F
from sklearn.metrics import pairwise_distances
from torch.optim.lr_scheduler import ReduceLROnPlateau
from tqdm.notebook import tqdm
import os
import sys
import time
import torch.nn as nn
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from stream_generators.mit_bih import MITBIH
from utils.MatplotlibUtils import reduce_dims_and_plot
from network.auto_encoder import AutoEncoder
from losses.knn_loss import KNNLoss, ClassificationKNNLoss
from sklearn.cluster import DBSCAN
from sklearn.mixture import GaussianMixture
from soft_decision_tree.sdt_model import SDT
from sklearn.metrics import davies_bouldin_score

In [2]:
k = 8
tree_depth = 8
batch_size = 512
device = 'cuda'
train_data_path = r'/mnt/qnap/ekosman/mitbih_train.csv'
test_data_path = r'/mnt/qnap/ekosman/mitbih_test.csv'

In [3]:
train_data_iter = torch.utils.data.DataLoader(MITBIH(train_data_path),
                                             batch_size=batch_size,
                                             shuffle=True,
                                             num_workers=1,
                                             pin_memory=True,
                                             drop_last=True)

test_data_iter = torch.utils.data.DataLoader(MITBIH(test_data_path),
                                             batch_size=batch_size,
                                             shuffle=True,
                                             num_workers=1,
                                             pin_memory=True)

In [4]:
class ConvBlock(nn.Module):
    def __init__(self):
        super(ConvBlock, self).__init__()
        self.conv1 = nn.Conv1d(32, 32, kernel_size=5, stride=1, padding=2)
        self.conv2 = nn.Conv1d(32, 32, kernel_size=5, stride=1, padding=2)
        self.relu1 = nn.ReLU()
        self.relu2 = nn.ReLU()
        self.pool = nn.MaxPool1d(kernel_size=5, stride=2)
        
    def forward(self, x):
        y = x
        y = self.conv1(y)
        y = self.relu1(y)
        y = self.conv2(y)
        y = y + x
        y = self.relu2(y)
        y = self.pool(y)
        return y


class ECGModel(nn.Module):
    def __init__(self):
        super(ECGModel, self).__init__()
        self.conv = nn.Conv1d(1, 32, kernel_size=5, stride=1, padding=1)
        self.block1 = ConvBlock()
        self.block2 = ConvBlock()
        self.block3 = ConvBlock()
        self.block4 = ConvBlock()
        self.block5 = ConvBlock()
        self.fc1 = nn.Linear(64, 32)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(32, 5)

    def forward(self, x, return_interm=False):
        x = self.conv(x)
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x)
        x = self.block5(x)
        interm = x.flatten(1)
        x = self.fc1(interm)
        x = self.relu(x)
        x = self.fc2(x)
        
        if return_interm:
            return x, interm
        
        return x

In [5]:
knn_crt = ClassificationKNNLoss(k=k).to(device)

def train(model, loader, optimizer, device):
    model.train()

    total_loss = 0
    for iteration, (batch, target) in enumerate(loader):
        batch = batch.to(device)
        target = target.to(device)
        outputs, interm = model(batch, return_interm=True)
        mse_loss = F.cross_entropy(outputs, target)
        mse_loss = mse_loss.sum(dim=-1).mean()
        try:
            knn_loss = knn_crt(interm, target)
            if torch.isinf(knn_loss):
                knn_loss = torch.tensor(0).to(device)
        except ValueError:
            knn_loss = torch.tensor(0).to(device)
        loss = mse_loss + knn_loss
        total_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if iteration % log_every == 0:
            print(f"Epoch {epoch} / {epochs} | iteration {iteration} / {len(loader)} | Total Loss: {loss.item()} | KNN Loss: {knn_loss.item()} | CLS Loss: {mse_loss.item()}")

    return total_loss / len(loader)

@torch.no_grad()
def test(model, loader, device):
    model.eval()
    
    correct = 0
    for iteration, (batch, target) in enumerate(loader):
        batch = batch.to(device)
        target = target.to(device)
        y_pred = model(batch).argmax(dim=-1)
        correct += y_pred.eq(target.view(-1).data).sum()
    
    return correct / len(loader.dataset)

In [6]:
epochs = 200
lr = 1e-3
log_every = 10

model = ECGModel().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
num_params = sum([p.numel() for p in model.parameters()])
print(f'#Params: {num_params}')

#Params: 53957


In [7]:
best_valid_acc = 0
losses = []
train_accs = []
val_accs = []
for epoch in range(1, epochs + 1):
    loss = train(model, train_data_iter, optimizer, device)
#     print(f"Loss: {loss} =============================")
    losses.append(loss)
    train_acc = test(model, train_data_iter, device)
    train_accs.append(train_acc)
    valid_acc = test(model, test_data_iter, device)
    val_accs.append(valid_acc)
    if valid_acc > best_valid_acc:
        best_valid_acc = valid_acc
    if epoch % 1 == 0:
        print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, '
              f'Train: {train_acc:.4f}, Valid: {valid_acc:.4f}, '
              f'Best: {best_valid_acc:.4f}')

Epoch 1 / 200 | iteration 0 / 171 | Total Loss: 7.258726119995117 | KNN Loss: 5.58424711227417 | CLS Loss: 1.6744791269302368
Epoch 1 / 200 | iteration 10 / 171 | Total Loss: 4.858896255493164 | KNN Loss: 4.17528772354126 | CLS Loss: 0.68360835313797
Epoch 1 / 200 | iteration 20 / 171 | Total Loss: 3.323235511779785 | KNN Loss: 2.731367588043213 | CLS Loss: 0.5918678641319275
Epoch 1 / 200 | iteration 30 / 171 | Total Loss: 3.1718411445617676 | KNN Loss: 2.56550669670105 | CLS Loss: 0.606334388256073
Epoch 1 / 200 | iteration 40 / 171 | Total Loss: 3.0967538356781006 | KNN Loss: 2.5365593433380127 | CLS Loss: 0.5601944327354431
Epoch 1 / 200 | iteration 50 / 171 | Total Loss: 2.9649579524993896 | KNN Loss: 2.501180410385132 | CLS Loss: 0.46377745270729065
Epoch 1 / 200 | iteration 60 / 171 | Total Loss: 3.0078890323638916 | KNN Loss: 2.502568006515503 | CLS Loss: 0.5053209662437439
Epoch 1 / 200 | iteration 70 / 171 | Total Loss: 2.9528684616088867 | KNN Loss: 2.4429826736450195 | CLS 

Epoch 4 / 200 | iteration 80 / 171 | Total Loss: 2.566361427307129 | KNN Loss: 2.463594675064087 | CLS Loss: 0.1027667224407196
Epoch 4 / 200 | iteration 90 / 171 | Total Loss: 2.593855142593384 | KNN Loss: 2.463956356048584 | CLS Loss: 0.12989874184131622
Epoch 4 / 200 | iteration 100 / 171 | Total Loss: 2.6004343032836914 | KNN Loss: 2.4911108016967773 | CLS Loss: 0.10932353138923645
Epoch 4 / 200 | iteration 110 / 171 | Total Loss: 2.61438250541687 | KNN Loss: 2.477689266204834 | CLS Loss: 0.13669322431087494
Epoch 4 / 200 | iteration 120 / 171 | Total Loss: 2.604036569595337 | KNN Loss: 2.478548049926758 | CLS Loss: 0.12548844516277313
Epoch 4 / 200 | iteration 130 / 171 | Total Loss: 2.550182819366455 | KNN Loss: 2.457052230834961 | CLS Loss: 0.09313064813613892
Epoch 4 / 200 | iteration 140 / 171 | Total Loss: 2.5695672035217285 | KNN Loss: 2.4919986724853516 | CLS Loss: 0.07756859809160233
Epoch 4 / 200 | iteration 150 / 171 | Total Loss: 2.602182149887085 | KNN Loss: 2.46675968

Epoch 7 / 200 | iteration 160 / 171 | Total Loss: 2.5054526329040527 | KNN Loss: 2.4551796913146973 | CLS Loss: 0.050273049622774124
Epoch 7 / 200 | iteration 170 / 171 | Total Loss: 2.5363872051239014 | KNN Loss: 2.4591224193573 | CLS Loss: 0.0772646889090538
Epoch: 007, Loss: 2.5362, Train: 0.9793, Valid: 0.9762, Best: 0.9762
Epoch 8 / 200 | iteration 0 / 171 | Total Loss: 2.514387607574463 | KNN Loss: 2.455143690109253 | CLS Loss: 0.05924391746520996
Epoch 8 / 200 | iteration 10 / 171 | Total Loss: 2.5086286067962646 | KNN Loss: 2.436633348464966 | CLS Loss: 0.07199514657258987
Epoch 8 / 200 | iteration 20 / 171 | Total Loss: 2.554171562194824 | KNN Loss: 2.47037935256958 | CLS Loss: 0.08379209041595459
Epoch 8 / 200 | iteration 30 / 171 | Total Loss: 2.537275791168213 | KNN Loss: 2.4291815757751465 | CLS Loss: 0.10809414088726044
Epoch 8 / 200 | iteration 40 / 171 | Total Loss: 2.5084729194641113 | KNN Loss: 2.450446367263794 | CLS Loss: 0.05802658945322037
Epoch 8 / 200 | iteratio

Epoch 11 / 200 | iteration 50 / 171 | Total Loss: 2.452906370162964 | KNN Loss: 2.4076926708221436 | CLS Loss: 0.04521365463733673
Epoch 11 / 200 | iteration 60 / 171 | Total Loss: 2.5393195152282715 | KNN Loss: 2.4924399852752686 | CLS Loss: 0.046879444271326065
Epoch 11 / 200 | iteration 70 / 171 | Total Loss: 2.507828712463379 | KNN Loss: 2.4357590675354004 | CLS Loss: 0.07206976413726807
Epoch 11 / 200 | iteration 80 / 171 | Total Loss: 2.4950079917907715 | KNN Loss: 2.415837287902832 | CLS Loss: 0.07917068898677826
Epoch 11 / 200 | iteration 90 / 171 | Total Loss: 2.5553457736968994 | KNN Loss: 2.4509036540985107 | CLS Loss: 0.10444211214780807
Epoch 11 / 200 | iteration 100 / 171 | Total Loss: 2.5299899578094482 | KNN Loss: 2.4251089096069336 | CLS Loss: 0.10488102585077286
Epoch 11 / 200 | iteration 110 / 171 | Total Loss: 2.478713274002075 | KNN Loss: 2.441230297088623 | CLS Loss: 0.03748295456171036
Epoch 11 / 200 | iteration 120 / 171 | Total Loss: 2.5011115074157715 | KNN Lo

Epoch 14 / 200 | iteration 120 / 171 | Total Loss: 2.450160503387451 | KNN Loss: 2.4191582202911377 | CLS Loss: 0.031002182513475418
Epoch 14 / 200 | iteration 130 / 171 | Total Loss: 2.490746021270752 | KNN Loss: 2.4164295196533203 | CLS Loss: 0.0743165984749794
Epoch 14 / 200 | iteration 140 / 171 | Total Loss: 2.510970115661621 | KNN Loss: 2.420605421066284 | CLS Loss: 0.09036476165056229
Epoch 14 / 200 | iteration 150 / 171 | Total Loss: 2.470524311065674 | KNN Loss: 2.4175517559051514 | CLS Loss: 0.05297257751226425
Epoch 14 / 200 | iteration 160 / 171 | Total Loss: 2.4557316303253174 | KNN Loss: 2.3701064586639404 | CLS Loss: 0.08562511205673218
Epoch 14 / 200 | iteration 170 / 171 | Total Loss: 2.411316156387329 | KNN Loss: 2.3702523708343506 | CLS Loss: 0.04106369614601135
Epoch: 014, Loss: 2.4797, Train: 0.9843, Valid: 0.9812, Best: 0.9812
Epoch 15 / 200 | iteration 0 / 171 | Total Loss: 2.4774720668792725 | KNN Loss: 2.4158856868743896 | CLS Loss: 0.06158629059791565
Epoch 15

...

Epoch: 185, Loss: 2.4103, Train: 0.9980, Valid: 0.9878, Best: 0.9878
Epoch 186 / 200 | iteration 0 / 171 | Total Loss: 2.4008283615112305 | KNN Loss: 2.385937213897705 | CLS Loss: 0.014891088008880615
Epoch 186 / 200 | iteration 10 / 171 | Total Loss: 2.3708126544952393 | KNN Loss: 2.3694918155670166 | CLS Loss: 0.001320941955782473
Epoch 186 / 200 | iteration 20 / 171 | Total Loss: 2.4096171855926514 | KNN Loss: 2.3974850177764893 | CLS Loss: 0.012132060714066029
Epoch 186 / 200 | iteration 30 / 171 | Total Loss: 2.3881924152374268 | KNN Loss: 2.38106107711792 | CLS Loss: 0.007131386548280716
Epoch 186 / 200 | iteration 40 / 171 | Total Loss: 2.362730026245117 | KNN Loss: 2.3547840118408203 | CLS Loss: 0.00794593058526516
Epoch 186 / 200 | iteration 50 / 171 | Total Loss: 2.391960382461548 | KNN Loss: 2.387556314468384 | CLS Loss: 0.004404039587825537
Epoch 186 / 200 | iteration 60 / 171 | Total Loss: 2.3919930458068848 | KNN Loss: 2.3834145069122314 | CLS Loss: 0.008578580804169178
E

Epoch 189 / 200 | iteration 60 / 171 | Total Loss: 2.425607681274414 | KNN Loss: 2.422868013381958 | CLS Loss: 0.00273971538990736
Epoch 189 / 200 | iteration 70 / 171 | Total Loss: 2.403491258621216 | KNN Loss: 2.3969478607177734 | CLS Loss: 0.006543281488120556
Epoch 189 / 200 | iteration 80 / 171 | Total Loss: 2.3887836933135986 | KNN Loss: 2.3818166255950928 | CLS Loss: 0.006966956425458193
Epoch 189 / 200 | iteration 90 / 171 | Total Loss: 2.432523012161255 | KNN Loss: 2.4093360900878906 | CLS Loss: 0.023186897858977318
Epoch 189 / 200 | iteration 100 / 171 | Total Loss: 2.386491537094116 | KNN Loss: 2.3740074634552 | CLS Loss: 0.012484166771173477
Epoch 189 / 200 | iteration 110 / 171 | Total Loss: 2.426100492477417 | KNN Loss: 2.4050347805023193 | CLS Loss: 0.021065719425678253
Epoch 189 / 200 | iteration 120 / 171 | Total Loss: 2.386218309402466 | KNN Loss: 2.384631872177124 | CLS Loss: 0.0015865350142121315
Epoch 189 / 200 | iteration 130 / 171 | Total Loss: 2.4077579975128174

Epoch 192 / 200 | iteration 120 / 171 | Total Loss: 2.3859593868255615 | KNN Loss: 2.378988742828369 | CLS Loss: 0.006970677059143782
Epoch 192 / 200 | iteration 130 / 171 | Total Loss: 2.409832715988159 | KNN Loss: 2.38132905960083 | CLS Loss: 0.028503581881523132
Epoch 192 / 200 | iteration 140 / 171 | Total Loss: 2.4100027084350586 | KNN Loss: 2.4044291973114014 | CLS Loss: 0.005573393311351538
Epoch 192 / 200 | iteration 150 / 171 | Total Loss: 2.433279514312744 | KNN Loss: 2.4250168800354004 | CLS Loss: 0.00826265662908554
Epoch 192 / 200 | iteration 160 / 171 | Total Loss: 2.4009337425231934 | KNN Loss: 2.390872001647949 | CLS Loss: 0.010061747394502163
Epoch 192 / 200 | iteration 170 / 171 | Total Loss: 2.368534564971924 | KNN Loss: 2.3657431602478027 | CLS Loss: 0.0027914096135646105
Epoch: 192, Loss: 2.4036, Train: 0.9964, Valid: 0.9853, Best: 0.9878
Epoch 193 / 200 | iteration 0 / 171 | Total Loss: 2.4220030307769775 | KNN Loss: 2.418938159942627 | CLS Loss: 0.003064960474148

Epoch: 195, Loss: 2.4044, Train: 0.9959, Valid: 0.9861, Best: 0.9878
Epoch 196 / 200 | iteration 0 / 171 | Total Loss: 2.382582664489746 | KNN Loss: 2.3770408630371094 | CLS Loss: 0.005541868507862091
Epoch 196 / 200 | iteration 10 / 171 | Total Loss: 2.387298345565796 | KNN Loss: 2.3784537315368652 | CLS Loss: 0.008844670839607716
Epoch 196 / 200 | iteration 20 / 171 | Total Loss: 2.4020190238952637 | KNN Loss: 2.400002956390381 | CLS Loss: 0.0020159974228590727
Epoch 196 / 200 | iteration 30 / 171 | Total Loss: 2.4295778274536133 | KNN Loss: 2.4134514331817627 | CLS Loss: 0.016126492992043495
Epoch 196 / 200 | iteration 40 / 171 | Total Loss: 2.3838071823120117 | KNN Loss: 2.369483709335327 | CLS Loss: 0.014323582872748375
Epoch 196 / 200 | iteration 50 / 171 | Total Loss: 2.4163501262664795 | KNN Loss: 2.4147579669952393 | CLS Loss: 0.0015922078164294362
Epoch 196 / 200 | iteration 60 / 171 | Total Loss: 2.3970820903778076 | KNN Loss: 2.391136884689331 | CLS Loss: 0.0059452550485730

Epoch 199 / 200 | iteration 60 / 171 | Total Loss: 2.4184951782226562 | KNN Loss: 2.4127755165100098 | CLS Loss: 0.005719643551856279
Epoch 199 / 200 | iteration 70 / 171 | Total Loss: 2.3907227516174316 | KNN Loss: 2.373054027557373 | CLS Loss: 0.017668629065155983
Epoch 199 / 200 | iteration 80 / 171 | Total Loss: 2.435577392578125 | KNN Loss: 2.432734966278076 | CLS Loss: 0.002842331537976861
Epoch 199 / 200 | iteration 90 / 171 | Total Loss: 2.37748384475708 | KNN Loss: 2.372047185897827 | CLS Loss: 0.005436714738607407
Epoch 199 / 200 | iteration 100 / 171 | Total Loss: 2.427713632583618 | KNN Loss: 2.4072234630584717 | CLS Loss: 0.02049008198082447
Epoch 199 / 200 | iteration 110 / 171 | Total Loss: 2.403379440307617 | KNN Loss: 2.3833584785461426 | CLS Loss: 0.020020943135023117
Epoch 199 / 200 | iteration 120 / 171 | Total Loss: 2.404508113861084 | KNN Loss: 2.400773286819458 | CLS Loss: 0.0037347835022956133
Epoch 199 / 200 | iteration 130 / 171 | Total Loss: 2.395252227783203

In [8]:
test(model, test_data_iter, device)

tensor(0.9863, device='cuda:0')

In [9]:
plt.figure()
plt.plot(losses, label='train loss')
plt.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
plt.figure()
plt.plot(train_accs, label='train accuracy')
plt.plot(val_accs, label='validation accuracy')
plt.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
test_samples = torch.tensor([])
projections = torch.tensor([])
labels = torch.tensor([])

with torch.no_grad():
    for x, y in tqdm(test_data_iter):
        test_samples = torch.cat([test_samples, x])
        labels = torch.cat([labels, y])
        x = x.to(device)
        _, interm = model(x, True)
        projections = torch.cat([projections, interm.detach().cpu().flatten(1)])

  0%|          | 0/43 [00:00<?, ?it/s]

In [12]:
distances = pairwise_distances(projections)
# distances = np.triu(distances)
distances_f = distances.flatten()

plt.matshow(distances)
plt.colorbar()
plt.figure()
plt.hist(distances_f[distances_f > 0], bins=1000)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
clusters = DBSCAN(eps=2, min_samples=10).fit_predict(projections)

In [14]:
print(f"Number of inliers: {sum(clusters != -1) / len(clusters)}")

Number of inliers: 0.9101457219862044


In [15]:
perplexity = 100
p = reduce_dims_and_plot(projections[clusters != -1],
                         y=clusters[clusters != -1],
                         title=f'perplexity: {perplexity}',
                         file_name=None,
                         perplexity=perplexity,
                         library='Multicore-TSNE',
                         perform_PCA=False,
                         projected=None,
                         figure_type='2d',
                         show_figure=True,
                         close_figure=False,
                         text=None)    

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Train a Soft-Decision-Tree given the self-labels

## Prepare the dataset

In [16]:
tree_dataset = list(zip(test_samples.flatten(1)[clusters!=-1], clusters[clusters != -1]))
batch_size = 512
tree_loader = torch.utils.data.DataLoader(tree_dataset, batch_size=batch_size, shuffle=True)

# Define how we prune the weights of a node

In [17]:
def prune_node(node_weights, factor=1):
    w = node_weights.cpu().detach().numpy()
    mean_ = np.mean(w)
    std_ = np.std(w)
    node_weights[((mean_ - std_ * factor) < node_weights) & (node_weights < (mean_ + std_ * factor))] = 0
    return node_weights

def prune_node_keep(node_weights, keep=4):
    w = node_weights.cpu().detach().numpy()
    throw_idx = np.argsort(abs(w))[:-keep]
    node_weights[throw_idx] = 0
    return node_weights

def prune_tree(tree_, factor):
    new_weights = tree_.inner_nodes.weight.clone()
    for i in range(new_weights.shape[0]):
        res = prune_node_keep(new_weights[i, :], factor)
        new_weights[i, :] = res

    with torch.no_grad():
        tree_.inner_nodes.weight.copy_(new_weights)
        
def sparseness(x):
    s = []
    for i in range(x.shape[0]):
        x_ = x[i, :]
        sp = (len(x_) - torch.norm(x_, 0).item()) / len(x_)
        s.append(sp)
    return np.mean(s)

def compute_regularization_by_level(tree):
    total_reg = 0
    for i in range(tree.inner_nodes.weight.shape[0]):
        cur_level = np.floor(np.log2(i+1))
        node_reg = torch.norm(tree.inner_nodes.weight[i].view(-1), 2)
        total_reg += 2**(-cur_level) * node_reg
    return total_reg

def show_sparseness(tree):
    avg_sp = sparseness(tree.inner_nodes.weight)
    print(f"Average sparseness: {avg_sp}")
    layer = 0
    sps = []
    for i in range(tree.inner_nodes.weight.shape[0]):
        cur_layer = int(np.floor(np.log2(i+1)))
        if cur_layer != layer:
            print(f"layer {layer}: {np.mean(sps)}")
            sps = []
            layer = cur_layer

        x_ = tree.inner_nodes.weight[i, :]
        sp = (len(x_) - torch.norm(x_, 0).item()) / len(x_)
        sps.append(sp)
        
    return avg_sp

## Training configurations

In [18]:
def do_epoch(model, loader, device, log_interval, losses, accs, epoch, iteration):
    model = model.train()
    for batch_idx, (data, target) in enumerate(loader):
        iteration += 1
        data, target = data.to(device), target.to(device)

        output, penalty = tree.forward(data)

        # Loss
        loss_tree = criterion(output, target.view(-1))

        # Penalty
        loss_tree += penalty

        # Sparse regularization
#         fc_params = torch.cat([x.view(-1) for x in tree.inner_nodes.parameters()])
#         regularization = sparsity_lamda * torch.norm(fc_params, 2)
        regularization = sparsity_lamda * compute_regularization_by_level(tree)
        loss = loss_tree

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        losses.append(loss.item())

        pred = output.data.max(1)[1]
        correct = pred.eq(target.view(-1).data).sum()
        accs.append(correct.item() / data.size()[0])

        # Print training status
        if batch_idx % log_interval == 0:
            print(f"Epoch: {epoch:02d} | Batch: {batch_idx:03d} / {len(loader):03d} | Total loss: {loss.item():.3f} | Reg loss: {regularization.item():.3f} | Tree loss: {loss_tree.item():.3f} | Accuracy: {correct.item() / data.size()[0]:03f} | {round((time.time() - start_time) / iteration, 3)} sec/iter")
            
    return iteration


In [19]:
lr = 5e-3
weight_decay = 5e-4
sparsity_lamda = 2e-3
epochs = 400
log_interval = 100
use_cuda = device != 'cpu'

In [20]:
tree = SDT(input_dim=test_samples.shape[2], output_dim=len(set(clusters)) - 1, depth=tree_depth, lamda=1e-3, use_cuda=use_cuda)
optimizer = torch.optim.Adam(tree.parameters(),
                                 lr=lr,
                                 weight_decay=weight_decay)
criterion = nn.CrossEntropyLoss()
tree = tree.to(device)

In [21]:
losses = []
accs = []
sparsity = []

In [22]:
start_time = time.time()
iteration = 0
for epoch in range(epochs):
    # Training
    avg_sp = show_sparseness(tree)
    sparsity.append(avg_sp)
    iteration = do_epoch(tree, tree_loader, device, log_interval, losses, accs, epoch, iteration)
    
    if epoch % 1 == 0:
        prune_tree(tree, factor=3)
        

Average sparseness: 0.0
layer 0: 0.0
layer 1: 0.0
layer 2: 0.0
layer 3: 0.0
layer 4: 0.0
layer 5: 0.0
layer 6: 0.0
Epoch: 00 | Batch: 000 / 039 | Total loss: 3.071 | Reg loss: 0.009 | Tree loss: 3.071 | Accuracy: 0.037109 | 0.245 sec/iter
Average sparseness: 0.9840425531914894
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
layer 5: 0.9840425531914894
layer 6: 0.9840425531914895
Epoch: 01 | Batch: 000 / 039 | Total loss: 2.984 | Reg loss: 0.005 | Tree loss: 2.984 | Accuracy: 0.326172 | 0.211 sec/iter
Average sparseness: 0.9840425531914894
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
layer 5: 0.9840425531914894
layer 6: 0.9840425531914895
Epoch: 02 | Batch: 000 / 039 | Total loss: 2.934 | Reg loss: 0.008 | Tree loss: 2.934 | Accuracy: 0.316406 | 0.209 sec/iter
Average sparseness: 0.9840425531914894
laye

Epoch: 23 | Batch: 000 / 039 | Total loss: 2.394 | Reg loss: 0.024 | Tree loss: 2.394 | Accuracy: 0.408203 | 0.205 sec/iter
Average sparseness: 0.9840425531914894
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
layer 5: 0.9840425531914894
layer 6: 0.9840425531914895
Epoch: 24 | Batch: 000 / 039 | Total loss: 2.455 | Reg loss: 0.025 | Tree loss: 2.455 | Accuracy: 0.341797 | 0.204 sec/iter
Average sparseness: 0.9840425531914894
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
layer 5: 0.9840425531914894
layer 6: 0.9840425531914895
Epoch: 25 | Batch: 000 / 039 | Total loss: 2.424 | Reg loss: 0.025 | Tree loss: 2.424 | Accuracy: 0.376953 | 0.204 sec/iter
Average sparseness: 0.9840425531914894
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4

Epoch: 46 | Batch: 000 / 039 | Total loss: 2.372 | Reg loss: 0.028 | Tree loss: 2.372 | Accuracy: 0.335938 | 0.205 sec/iter
Average sparseness: 0.9840425531914894
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
layer 5: 0.9840425531914894
layer 6: 0.9840425531914895
Epoch: 47 | Batch: 000 / 039 | Total loss: 2.272 | Reg loss: 0.029 | Tree loss: 2.272 | Accuracy: 0.392578 | 0.205 sec/iter
Average sparseness: 0.9840425531914894
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
layer 5: 0.9840425531914894
layer 6: 0.9840425531914895
Epoch: 48 | Batch: 000 / 039 | Total loss: 2.365 | Reg loss: 0.029 | Tree loss: 2.365 | Accuracy: 0.339844 | 0.205 sec/iter
Average sparseness: 0.9840425531914894
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4

Epoch: 69 | Batch: 000 / 039 | Total loss: 2.414 | Reg loss: 0.030 | Tree loss: 2.414 | Accuracy: 0.292969 | 0.204 sec/iter
Average sparseness: 0.9840425531914894
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
layer 5: 0.9840425531914894
layer 6: 0.9840425531914895
Epoch: 70 | Batch: 000 / 039 | Total loss: 2.290 | Reg loss: 0.030 | Tree loss: 2.290 | Accuracy: 0.333984 | 0.204 sec/iter
Average sparseness: 0.9840425531914894
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
layer 5: 0.9840425531914894
layer 6: 0.9840425531914895
Epoch: 71 | Batch: 000 / 039 | Total loss: 2.336 | Reg loss: 0.030 | Tree loss: 2.336 | Accuracy: 0.320312 | 0.204 sec/iter
Average sparseness: 0.9840425531914894
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4

Epoch: 92 | Batch: 000 / 039 | Total loss: 2.222 | Reg loss: 0.031 | Tree loss: 2.222 | Accuracy: 0.361328 | 0.203 sec/iter
Average sparseness: 0.9840425531914894
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
layer 5: 0.9840425531914894
layer 6: 0.9840425531914895
Epoch: 93 | Batch: 000 / 039 | Total loss: 2.330 | Reg loss: 0.031 | Tree loss: 2.330 | Accuracy: 0.312500 | 0.203 sec/iter
Average sparseness: 0.9840425531914894
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
layer 5: 0.9840425531914894
layer 6: 0.9840425531914895
Epoch: 94 | Batch: 000 / 039 | Total loss: 2.276 | Reg loss: 0.031 | Tree loss: 2.276 | Accuracy: 0.347656 | 0.203 sec/iter
Average sparseness: 0.9840425531914894
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4

...

Epoch: 299 | Batch: 000 / 039 | Total loss: 2.148 | Reg loss: 0.032 | Tree loss: 2.148 | Accuracy: 0.392578 | 0.203 sec/iter
Average sparseness: 0.9840425531914894
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
layer 5: 0.9840425531914894
layer 6: 0.9840425531914895
Epoch: 300 | Batch: 000 / 039 | Total loss: 2.200 | Reg loss: 0.032 | Tree loss: 2.200 | Accuracy: 0.376953 | 0.203 sec/iter
Average sparseness: 0.9840425531914894
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
layer 5: 0.9840425531914894
layer 6: 0.9840425531914895
Epoch: 301 | Batch: 000 / 039 | Total loss: 2.179 | Reg loss: 0.032 | Tree loss: 2.179 | Accuracy: 0.375000 | 0.203 sec/iter
Average sparseness: 0.9840425531914894
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
laye

Epoch: 322 | Batch: 000 / 039 | Total loss: 2.130 | Reg loss: 0.032 | Tree loss: 2.130 | Accuracy: 0.382812 | 0.204 sec/iter
Average sparseness: 0.9840425531914894
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
layer 5: 0.9840425531914894
layer 6: 0.9840425531914895
Epoch: 323 | Batch: 000 / 039 | Total loss: 2.135 | Reg loss: 0.032 | Tree loss: 2.135 | Accuracy: 0.384766 | 0.204 sec/iter
Average sparseness: 0.9840425531914894
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
layer 5: 0.9840425531914894
layer 6: 0.9840425531914895
Epoch: 324 | Batch: 000 / 039 | Total loss: 2.234 | Reg loss: 0.032 | Tree loss: 2.234 | Accuracy: 0.349609 | 0.204 sec/iter
Average sparseness: 0.9840425531914894
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
laye

Epoch: 345 | Batch: 000 / 039 | Total loss: 2.196 | Reg loss: 0.032 | Tree loss: 2.196 | Accuracy: 0.359375 | 0.204 sec/iter
Average sparseness: 0.9840425531914894
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
layer 5: 0.9840425531914894
layer 6: 0.9840425531914895
Epoch: 346 | Batch: 000 / 039 | Total loss: 2.127 | Reg loss: 0.032 | Tree loss: 2.127 | Accuracy: 0.402344 | 0.204 sec/iter
Average sparseness: 0.9840425531914894
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
layer 5: 0.9840425531914894
layer 6: 0.9840425531914895
Epoch: 347 | Batch: 000 / 039 | Total loss: 2.219 | Reg loss: 0.032 | Tree loss: 2.219 | Accuracy: 0.367188 | 0.204 sec/iter
Average sparseness: 0.9840425531914894
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
laye

Epoch: 368 | Batch: 000 / 039 | Total loss: 2.139 | Reg loss: 0.032 | Tree loss: 2.139 | Accuracy: 0.398438 | 0.204 sec/iter
Average sparseness: 0.9840425531914894
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
layer 5: 0.9840425531914894
layer 6: 0.9840425531914895
Epoch: 369 | Batch: 000 / 039 | Total loss: 2.168 | Reg loss: 0.032 | Tree loss: 2.168 | Accuracy: 0.382812 | 0.204 sec/iter
Average sparseness: 0.9840425531914894
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
layer 5: 0.9840425531914894
layer 6: 0.9840425531914895
Epoch: 370 | Batch: 000 / 039 | Total loss: 2.166 | Reg loss: 0.032 | Tree loss: 2.166 | Accuracy: 0.388672 | 0.204 sec/iter
Average sparseness: 0.9840425531914894
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
laye

Epoch: 391 | Batch: 000 / 039 | Total loss: 2.142 | Reg loss: 0.032 | Tree loss: 2.142 | Accuracy: 0.394531 | 0.205 sec/iter
Average sparseness: 0.9840425531914894
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
layer 5: 0.9840425531914894
layer 6: 0.9840425531914895
Epoch: 392 | Batch: 000 / 039 | Total loss: 2.239 | Reg loss: 0.032 | Tree loss: 2.239 | Accuracy: 0.355469 | 0.205 sec/iter
Average sparseness: 0.9840425531914894
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
layer 5: 0.9840425531914894
layer 6: 0.9840425531914895
Epoch: 393 | Batch: 000 / 039 | Total loss: 2.181 | Reg loss: 0.032 | Tree loss: 2.181 | Accuracy: 0.367188 | 0.205 sec/iter
Average sparseness: 0.9840425531914894
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
laye

In [23]:
plt.figure(figsize=(10, 5))
plt.ylabel("Accuracy")
plt.xlabel('Iteration')
plt.plot(accs, label='Accuracy vs iteration')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [24]:
plt.figure()
plt.ylabel("Loss")
plt.xlabel('Iteration')
plt.plot(losses, label='Loss vs iteration')
# plt.yscale("log")
plt.show()

plt.figure()
weights = tree.inner_nodes.weight.cpu().detach().numpy().flatten()
plt.hist(weights, bins=500)
weights_std = np.std(weights)
weights_mean = np.mean(weights)
plt.axvline(weights_mean + weights_std, color='r')
plt.axvline(weights_mean - weights_std, color='r')
plt.title(f"Mean: {weights_mean}   |   STD: {weights_std}")
plt.yscale("log")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Tree Visualization

In [25]:
plt.figure(figsize=(15, 10), dpi=80)
avg_height, root = tree.visualize()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Average height: 6.6521739130434785


# Extract Rules

# Accumulate samples in the leaves

In [26]:
print(f"Number of patterns: {len(root.get_leaves())}")

Number of patterns: 46


In [27]:
method = 'greedy'

In [28]:
root.clear_leaves_samples()

with torch.no_grad():
    for batch_idx, (data, target) in enumerate(tree_loader):
        root.accumulate_samples(data, method)

/home/eitan.k/.local/lib/python3.6/site-packages/torch/nn/functional.py:1709: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


# Tighten boundaries

In [29]:
attr_names = [f"T_{i}" for i in range(test_samples.shape[2])]
leaves = root.get_leaves()
sum_comprehensibility = 0
comprehensibilities = []
for pattern_counter, leaf in enumerate(leaves):
    leaf.reset_path()
    leaf.tighten_with_accumulated_samples()
    conds = leaf.get_path_conditions(attr_names)
    print(f"============== Pattern {pattern_counter + 1} ==============")
    comprehensibilities.append(sum([cond.comprehensibility for cond in conds]))
    
print(f"Average comprehensibility: {np.mean(comprehensibilities)}")
print(f"std comprehensibility: {np.std(comprehensibilities)}")
print(f"var comprehensibility: {np.var(comprehensibilities)}")
print(f"minimum comprehensibility: {np.min(comprehensibilities)}")
print(f"maximum comprehensibility: {np.max(comprehensibilities)}")

============== Pattern 1 ==============
2292
============== Pattern 2 ==============
============== Pattern 3 ==============
============== Pattern 4 ==============
============== Pattern 5 ==============
============== Pattern 6 ==============
============== Pattern 7 ==============
============== Pattern 8 ==============
============== Pattern 9 ==============
============== Pattern 10 ==============
4
============== Pattern 11 ==============
15706
============== Pattern 12 ==============
105
============== Pattern 13 ==============
============== Pattern 14 ==============
============== Pattern 15 ==============
============== Pattern 16 ==============
============== Pattern 17 ==============
============== Pattern 18 ==============
============== Pattern 19 ==============
============== Pattern 20 ==============
============== Pattern 21 ==============
============== Pattern 22 ==============
============== Pattern 23 ==============
============== Pattern 24 ==============
========

/home/eitan.k/EntangledExplainableClustering/soft_decision_tree/sdt_model.py:14: RuntimeWarning: divide by zero encountered in log
  return np.log(1 / (1 - x))
